# 量子模拟器

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/mindquantum/zh_cn/mindspore_quantum_simulator.ipynb)&emsp;
[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/mindquantum/zh_cn/mindspore_quantum_simulator.py)&emsp;
[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/docs/mindquantum/docs/source_zh_cn/quantum_simulator.ipynb)

## 概述

搭建出量子线路后，我们需要指定一个后端来运行量子线路，在MindQuantum中，我们可以利用量子模拟器`Simulator`来对量子线路进行模拟运行。在本教程中我们声明一个两比特的`projectq`模拟器，并以此来简介模拟器的关键功能。

## 环境准备


导入本教程所依赖的模块。

In [1]:
import numpy as np                             # 导入numpy库并简写为np
from mindquantum.simulator import Simulator    # 从mindquantum.simulator中导入Simulator类
from mindquantum.core import X, H, RY          # 导入量子门H, X, RY

说明：

（1）numpy是一个功能强大的Python库，主要用于对多维数组执行计算，支持大量的维度数组与矩阵运算，此外也针对数组运算提供大量的数学函数库；

（2）mindquantum是量子-经典混合计算框架，支持多种量子神经网络的训练和推理；

（3）搭建的量子线路中所需执行的量子门需要从mindquantum.core模块中导入；

In [2]:
sim = Simulator('projectq', 2)   #声明一个两比特的projectq模拟器
sim                              #展示模拟器状态

projectq simulator with 2 qubits (little endian).
Current quantum state:
1¦00⟩

在MindQuantum中，我们可以在`mindquantum.simulator`模块导入模拟器。`Simulator`类可以接受三个参数：

- `backend`：所用到的模拟器名称，目前`mindquantum`支持`projectq`作为后端进行模拟。
- `n_qbuits`：模拟器所用到的比特数，也就是这里的2。
- `seed`：模拟器在运行随机性相关算法时的随机种子，默认为一个随机数，可以不用提供。

通过模拟器的输出结果我们可以发现，这是一个`projectq`的2比特模拟器，并且是little endian的。这里little endian的意思是，整个模拟器中，我们都是将比特序号小的比特放在量子态矢量的右边。接下来，输出还说明了模拟器当前所处的量子态是多少，且在模拟器初始化后，当前的量子态默认处于零态。注意，量子模拟器始终会维护一个内部的量子态，当我们作用量子门或者量子线路到模拟器上时，这个量子态会随即发生改变，而当我们只是想获取关于这个量子态的一些信息时，这个量子态则不会改变。这里就涉及到对量子模拟器的两类操作：

- 会改变量子态的操作，通常以`apply`开头，主要有如下几个
    - `apply_gate`: 作用一个量子门到模拟器上
    - `apply_circuit`: 作用一个量子线路到模拟器上
    - `apply_hamiltonian`: 将一个哈密顿量作用到模拟器上，注意，此后模拟器的量子态将不再是一个真的量子态
    - `set_qs`: 直接设置模拟器的当前量子态
    - `reset`: 重置模拟器的状态为|0⟩态
- 不会改变量子态的操作，通常以`get`开头，主要有如下几个
    - `get_qs`: 获取模拟器的当前量子态
    - `get_expectation`: 计算模拟器当前量子态关于某个观察量的期望值
    - `get_expectation_with_grad`: 跟上一个接口类似，只不过这个方法还会计算期望值关于参数化量子线路的梯度
    - `sampling`: 在当前量子态下，对给定的量子线路进行采样

下面我们简单学习模拟器的基本操作。

## 作用量子门和量子线路

In [3]:
sim = Simulator('projectq', 2)    #声明一个2比特的projectq模拟器
sim.apply_gate(H.on(0))           #作用一个Hadamard门到0号比特上
sim                               #输出量子模拟器的信息

projectq simulator with 2 qubits (little endian).
Current quantum state:
√2/2¦00⟩
√2/2¦01⟩

上面我们在量子模拟器的初态上作用了一个Hadamard门，并输出了演化过后的量子态。接下来我们生成一个参数化量子线路，并将其作用到当前的量子态上。

In [4]:
from mindquantum.core import Circuit   # 导入Circuit模块，用于搭建量子线路

circ = Circuit()                       #声明一个空的量子线路
circ += H.on(1)                        #向其中添加一个hadamard门，并作用到1号比特上
circ += RY('a').on(0)                  #向其中添加一个参数化的RY门，并作用到0号比特上
circ.svg()                             #绘制SVG格式的量子线路图片

In [5]:
sim.apply_circuit(circ, pr={'a': 1.234})  #作用一个量子线路，当线路是一个参数化量子线路时，我们还需要提供参数值。
sim

projectq simulator with 2 qubits (little endian).
Current quantum state:
0.11851349145283657¦00⟩
0.6971044056263442¦01⟩
0.11851349145283657¦10⟩
0.6971044056263442¦11⟩

在上面的代码中，我们先生成了一个参数化量子线路`circ`，随后我们将其作用到量子模拟器上，并通过传入字典的方式，将参数`a`设置为`1.234`。最后输出量子模拟器演化出来的量子态。

## 设置并获取模拟器状态

我们使用`get_qs(ket=False)`查看当前模拟器的状态，
参数ket是一个bool类型的数，它决定了当前模拟器的状态是否以ket的形式返回，ket=False时是以numpy.ndarray形式，ket=True时是以ket形式。默认ket=False。

In [6]:
print(sim.get_qs())  #查看模拟器状态,以numpy.ndarray形式返回结果

[0.11851349+0.j 0.69710441+0.j 0.11851349+0.j 0.69710441+0.j]


In [7]:
print(sim.get_qs(True))  #查看模拟器状态,以ket形式返回结果

0.11851349145283657¦00⟩
0.6971044056263442¦01⟩
0.11851349145283657¦10⟩
0.6971044056263442¦11⟩


在实际写代码过程中，我们常常需要将模拟器指定一个初始态开始演化，这个操作可以使用`set_qs()`实现。

例如，我们希望模拟器状态为

$$
\frac{\sqrt{3}}{3}|00⟩+\frac{\sqrt{6}}{3}|11⟩
$$

第一步：我们计算出目标状态的向量形式：

$$
\frac{\sqrt{3}}{3}|00⟩+\frac{\sqrt{6}}{3}|11⟩ =\frac{\sqrt{3}}{3}\times
\left(
\begin{array}{l}
1\\
0
\end{array}
\right)
\otimes
\left(
\begin{array}{l}
1\\
0
\end{array}
\right)+
\frac{\sqrt{6}}{3}\times
\left(
\begin{array}{l}
0\\
1
\end{array}
\right)\otimes
\left(
\begin{array}{l}
0\\
1
\end{array}
\right)= \frac{\sqrt{3}}{3}\times
\left(
\begin{array}{l}
1\\
0\\
0\\
0
\end{array}
\right)+
\frac{\sqrt{6}}{3}\times
\left(
\begin{array}{l}
0\\
0\\
0\\
1
\end{array}
\right)=
\left(
\begin{array}{l}
\frac{\sqrt{3}}{3}\\
0\\
0\\
\frac{\sqrt{6}}{3}
\end{array}
\right)
$$

第二步：我们将这个向量使用`set_qs()`赋值给模拟器，让其作为模拟器的状态：

In [8]:
sim.set_qs(np.array([3**0.5, 0, 0, 6**0.5]))        #设置模拟器状态，无需归一化
print(sim.get_qs(True))                             #查看模拟器状态

0.5773502691896258¦00⟩
0.816496580927726¦11⟩


通过`get_qs()`查看模拟器状态可以发现，当前模拟器状态即为我们希望设置的$\frac{\sqrt{3}}{3}|00⟩+\frac{\sqrt{6}}{3}|11⟩$。

在实际编程过程中，我们常常需要多次模拟电路，通过多开模拟器的方式会导致内存占用非常大，我们可以通过现有模拟器复位的方式来复用模拟器，从而减少内存消耗。

我们使用`reset()`来复位模拟器：

In [9]:
sim.reset()          #复位模拟器
print(sim.get_qs())  #查看模拟器状态

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]


可以发现，当前模拟器被复位成了初始的$1|00⟩$态，相当于一个全新的模拟器。

因此，我们可以根据自身所需的量子初态，设置对应的量子模拟器，并运行自定义的量子线路。赶紧动手运行你构造出的的第一个量子线路吧！

## 量子线路采样

线路采样是指对量子线路执行多次模拟测量，统计测量出各种结果出现的频次。**采样不会改变量子线路中的状态**。

`sampling(circuit, pr=None, shots=1, seed=None)`是`MindQuantum`中提供的对模拟器进行线路采样方法，它接受四个参数：

- `circuit (Circuit)`：希望进行采样的量子线路，注意，该线路中必须包含至少一个测量操作（即采样点）。
- `pr (Union[None, dict, ParameterResolver])`：parameter resolver，当 `circuit`是含参线路时，需要给出参数的值。
- `shots (int)`：采样的次数，默认为1。
- `seed`：采样时的随机种子，默认为一个随机数，可以不用提供。

In [10]:
# 不含参线路采样：
from mindquantum import Measure  # 引入测量门

circ = Circuit()                 # 初始化量子线路
circ += H.on(0)                  # H门作用在第0位量子比特
circ += X.on(1, 0)               # X门作用在第1位量子比特且受第0位量子比特控制
circ += Measure('q0').on(0)      # 在0号量子比特作用一个测量，并将该测量命名为'q0'
circ += Measure('q1').on(1)      # 在1号量子比特作用一个测量，并将该测量命名为'q1'
circ.svg()                       # 绘制SVG格式的量子线路图片

In [11]:
sim.reset()
result = sim.sampling(circ, shots=1000)  # 对上面定义的线路采样1000次
result

shots: 1000
Keys: q1 q0│0.00   0.128       0.256       0.384       0.512        0.64
───────────┼───────────┴───────────┴───────────┴───────────┴───────────┴
         00│▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
           │
         11│▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
           │
{'00': 488, '11': 512}

MindQuantum还提供了采样结果绘制SVG图的功能：

In [12]:
result.svg()  # 打印出测量结果的SVG格式

我们可以看到，采样1000中，'00'出现了499次，'11'出现了501次。我们搭建的线路实际上制备出了一个贝尔态$\frac{\sqrt{2}}{2}|00⟩+\frac{\sqrt{2}}{2}|11⟩$。直观上，我们可以看到对该状态进行测量得到'00'的概率为$\frac{1}{2}$,得到'11'的概率为$\frac{1}{2}$，采样结果符合概率，细微的误差是由模拟器噪声导致。

In [13]:
# 含参线路采样：
para_circ = Circuit()             # 初始化量子线路
para_circ += H.on(0)              # H门作用在第0位量子比特
para_circ += X.on(1, 0)           # X门作用在第1位量子比特且受第0位量子比特控制
para_circ += RY('theta').on(1)    # RY(theta)门作用在第2位量子比特
para_circ += Measure('0').on(0)   # 在0号量子比特作用一个测量，并将该测量命名为'q0'
para_circ += Measure('q1').on(1)  # 在1号量子比特作用一个测量，并将该测量命名为'q1'
para_circ.svg()                   # 绘制SVG格式的量子线路图片

In [14]:
sim.reset()
result = sim.sampling(para_circ, {'theta': 0},
                      shots=1000)  # 将上面定义的线路参数'theta'赋值为0采样1000次
result.svg()

我们可以看到，采样结果中'00'出现了495次，'11'出现了505次。事实上把RY门参数赋值为0，它即为我们熟悉的I门，相当于不对线路做任何操作，因此该采样线路与上面不含参线路本质是同一个，可以观察到二次采样结果几乎相同，符合预期结果。

想进一步学习如何对量子线路做测量操作，想了解采样结果分布的理论解释，请点击：[量子测量教程](https://gitee.com/buyulin/mindquantum/blob/master/tutorials/quantum_measurement.ipynb)。

若想查询更多关于MindQuantum的API，请点击：[https://mindspore.cn/mindquantum/](https://mindspore.cn/mindquantum/)。